In [16]:
import os
os.chdir('/home/bohdan/projects/glider/lux-ai')

In [1]:
import gym
from luxai_s2.env import LuxAI_S2
import matplotlib.pyplot as plt
import numpy as np
import os
from lux.kit import obs_to_game_state, GameState, EnvConfig

pygame 2.2.0 (SDL 2.0.22, Python 3.9.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [44]:
from luxai_s2.utils.utils import my_turn_to_place_factory


def interact(env, agents, steps):
    obs = env.reset(seed=41)
    step = 0
    # iterate until phase 1 ends
    while env.state.real_env_steps < 0:
        if step >= steps: break
        actions = {}
        for player in env.agents:
            o = obs[player]
            a = agents[player].early_setup(step, o)
            actions[player] = a
        step += 1
        obs, rewards, dones, infos = env.step(actions)

    return obs, rewards, dones, infos


class Agent():
    def __init__(self, player: str, env_cfg: EnvConfig) -> None:
        self.player = player
        self.opp_player = "player_1" if self.player == "player_0" else "player_0"
        np.random.seed(0)
        self.env_cfg: EnvConfig = env_cfg

    def early_setup(self, step: int, obs, remainingOverageTime: int = 60):
        if step == 0:
            # bid 0 to not waste resources bidding and declare as the default faction
            # you can bid -n to prefer going second or n to prefer going first in placement
            return dict(faction="AlphaStrike", bid=0)
        
        game_state = obs_to_game_state(step, self.env_cfg, obs)
        # factory placement period
        
        # how much water and metal you have in your starting pool to give to new factories
        factories_to_place = game_state.teams[self.player].factories_to_place
        # whether it is your turn to place a factory
        my_turn_to_place = my_turn_to_place_factory(game_state.teams[self.player].place_first, step)
        if factories_to_place > 0 and my_turn_to_place:
            # we will spawn our factory in a random location with 150 metal and water if it is our turn to place
            potential_spawns = np.array(list(zip(*np.where(obs["board"]["valid_spawns_mask"] == 1))))
            spawn_loc = potential_spawns[np.random.randint(0, len(potential_spawns))]
            return dict(spawn=spawn_loc, metal=150, water=150)
        return dict()

    def act(self, step: int, obs, remainingOverageTime: int = 60):
        actions = dict()
        game_state = obs_to_game_state(step, self.env_cfg, obs)
        return actions

In [45]:
env = LuxAI_S2()
obs = env.reset(42)


In [46]:
game_state = obs_to_game_state(0, env.env_cfg, obs['player_1'])

In [48]:
agents = {a: Agent(a, env.state.env_cfg) for a in env.agents}
obs, *_ = interact(env, agents, 2)


In [52]:
p_obs = obs['player_0']
p_obs = obs_to_game_state(2, env.env_cfg, p_obs)

In [53]:
p_obs.factories

{'player_0': {'factory_0': Factory(team_id=0, unit_id='factory_0', strain_id=0, power=1000, cargo=UnitCargo(ice=0, ore=0, water=150, metal=150), pos=array([18,  4]), env_cfg=EnvConfig(max_episode_length=1000, map_size=48, verbose=1, validate_action_space=True, max_transfer_amount=3000, MIN_FACTORIES=2, MAX_FACTORIES=5, CYCLE_LENGTH=50, DAY_LENGTH=30, UNIT_ACTION_QUEUE_SIZE=20, MAX_RUBBLE=100, FACTORY_RUBBLE_AFTER_DESTRUCTION=50, INIT_WATER_METAL_PER_FACTORY=150, INIT_POWER_PER_FACTORY=1000, MIN_LICHEN_TO_SPREAD=20, LICHEN_LOST_WITHOUT_WATER=1, LICHEN_GAINED_WITH_WATER=1, MAX_LICHEN_PER_TILE=100, POWER_PER_CONNECTED_LICHEN_TILE=1, LICHEN_WATERING_COST_FACTOR=10, BIDDING_SYSTEM=True, FACTORY_PROCESSING_RATE_WATER=100, ICE_WATER_RATIO=4, FACTORY_PROCESSING_RATE_METAL=50, ORE_METAL_RATIO=5, FACTORY_CHARGE=50, FACTORY_WATER_CONSUMPTION=1, POWER_LOSS_FACTOR=0.5, ROBOTS={'LIGHT': UnitConfig(METAL_COST=10, POWER_COST=50, CARGO_SPACE=100, BATTERY_CAPACITY=150, CHARGE=1, INIT_POWER=50, MOVE_COST